In [1]:
from line_profiler import LineProfiler

In [2]:
%load_ext line_profiler

## Simple software division of two 32 bit integers

Source: http://bearcave.com/software/divide.htm

/*
   Copyright stuff

   Use of this program, for any purpose, is granted the author,
   Ian Kaplan, as long as this copyright notice is included in
   the source code or any source code derived from this program.
   The user assumes all responsibility for using this code.

   Ian Kaplan, October 1996

*/


In [1]:
def unsigned_divide(dividend: 'uint', divisor: 'uint') -> ('uint', 'uint'):
    t       : 'uint' = None
    num_bits: 'uint' = None
    q       : 'uint' = None
    bit     : 'uint' = None
    d       : 'uint' = None
    i       : 'int'  = None
    
    remainder : 'uint' = 0
    quotient  : 'uint' = 0

    if   (divisor == 0       ): pass
    elif (divisor  > dividend): remainder = dividend
    elif (divisor == dividend): quotient = 1
    else:
        num_bits = 32
        while (remainder < divisor):
            bit       = (dividend & 0x80000000) >> 31 # grab the highest bit in dividend
            remainder = (remainder << 1) | bit        # shift up remained and move bit into lowest position
            d         = dividend                      # store for later
            dividend  = dividend << 1                 # shift up dividend, putting the next highest bit in highest position
            num_bits -= 1                             # count down the number of bits remaining in dividend
            
        # The loop, above, always goes one iteration too far.
        # To avoid inserting an "if" statement inside the loop
        # the last iteration is simply reversed.
        dividend  = d              # restore dividend to previous value
        remainder = remainder >> 1 # shift remainder back down, erasing the previously lowest bit
        num_bits += 1              # undo the last counter update
        
        for i in range(num_bits):
            bit       = (dividend & 0x80000000) >> 31 # see above
            remainder = (remainder << 1) | bit        # see above
            t         = remainder - divisor           # remainder is larger than divisor, so get positive difference
            q         = not ((t & 0x80000000) >> 31)  # grab negated highest bit of previous division
            dividend  = dividend << 1                 # see above
            quotient  = (quotient << 1) | q           # 
            if (q): remainder = t                     # if highest bit of t == 0, remainder = t
    
    return quotient, remainder

In [29]:
numerator: 'uint' = 150 # aka dividend
divisor  : 'uint' = 4
remainder: 'uint' = 0
quotient : 'uint' = 0
t = q = restore = bit = num_bits = i = None; del t, q, restore, bit, num_bits, i
if   (divisor == 0        ): raise ZeroDivisionError
elif (divisor  > numerator): remainder = numerator
elif (divisor == numerator): quotient  = 1
else:
    num_bits:'uint' = 32
    while (remainder < divisor):
        bit       = (numerator & 0x80000000) >> 31 # grab the 32th bit in numerator
        remainder = (remainder << 1) | bit         # shift up remained and move bit into lowest position
        restore   = numerator                      # store for later
        numerator = (numerator << 1) & 0xFFFFFFFF  # shift up numerator, putting the next highest bit in 32th position *1
        num_bits -= 1                              # count down the number of bits remaining in numerator
    # The loop, above, always goes one iteration too far.
    # To avoid inserting an "if" statement inside the loop, the last iteration is simply reversed.
    numerator = restore        # restore numerator to previous value
    remainder = remainder >> 1 # shift remainder back down, erasing the previously lowest bit
    num_bits += 1              # undo the last counter update
    
    print(bin(numerator), bin(divisor), bin(remainder))
    print()
    
    if 1:
        for i in range(num_bits):
            bit       = (numerator & 0x80000000) >> 31 # see above
            numerator = (numerator << 1) & 0xFFFFFFFF  # see above
            remainder = (remainder << 1) | bit         # see above
            if remainder >= divisor:
                print(i, bin(remainder),'-', bin(divisor),'=', bin(remainder - divisor))
                remainder = remainder - divisor            # 
                quotient  = (quotient << 1) | 1            # shift in a 1
            else:
                quotient  = (quotient << 1)                # shift in a zero
                print(i, )
            print(bin(numerator), bin(quotient))
    else:
        for i in range(num_bits):
            bit       = (numerator & 0x80000000) >> 31 # see above
            numerator = (numerator << 1) & 0xFFFFFFFF  # see above
            remainder = (remainder << 1) | bit         # see above
            t         = remainder - divisor            # 
            q         = t >= 0                         # check if result of subtraction has gone below zero *2
            quotient  = (quotient << 1) | q            # shift up quotient and store negated highest bit of previous division
            print(i, bin(remainder),'-', bin(divisor),'=', t, f'({bin(t)})', q)
            print(bin(numerator), bin(quotient))
            if (q): remainder = t                      # if remainder >= divisor: remainder = t

quotient, remainder

0b1011000000000000000000000000000 0b100 0b10

0 0b100 - 0b100 = 0b0
0b10110000000000000000000000000000 0b1
1
0b1100000000000000000000000000000 0b10
2
0b11000000000000000000000000000000 0b100
3 0b101 - 0b100 = 0b1
0b10000000000000000000000000000000 0b1001
4
0b0 0b10010
5 0b110 - 0b100 = 0b10
0b0 0b100101


(37, 2)

In [64]:
150 // 4, 150 % 4

(37, 2)

https://www.youtube.com/watch?v=VKemv9u40gc

*1 Python support arbitrary sized integers, so shifting up doesn't actually remove the highest bit, but instead just moves in a 0 at the right, effectively doubling the number. Since we're grabbing the 32th bit every time this doesn't really matter, but should we want this algorithm to work with arbitrary sized integers instead of fixed 32 bit, then this need to be changed. To "fix this we could replace `numerator << 1` with `(numerator << 1) & 0xFFFFFFFF`.

---

*2 In a C program, we would be working with unsigned numbers here, which wrap around to max int when going below zero. So when we're testing the highest bit for 0 or 1, we really asking if we've wrapped around, or in other words, gone negative. So since we're working with signed integers in python, `not ((t & 0x80000000) >> 31)` really is equivalent to `t >= 0`, which in turn is equivalent to checking if `remainder >= divisor`.

---



#### signed divide

In [15]:
def signed_divide(dividend: 'int', divisor: 'int') -> ('int', 'int'):
    dend: 'uint' = abs(dividend)
    dor : 'uint' = abs(divisor)
    q   : 'uint' = None
    r   : 'uint' = None
    
    remainder : 'int' = 0
    quotient  : 'int' = 0

    q, r = unsigned_divide(dend, dor)

#     /* the sign of the remainder is the same as the sign of the dividend
#      and the quotient is negated if the signs of the operands are
#      opposite */
    quotient = q
    if (dividend < 0):
        remainder = -r;
        if (divisor > 0):
            quotient = -q;
    else: # /* positive dividend */
        remainder = r;
        if (divisor < 0):
            quotient = -q;
    return quotient, remainder

In [18]:
signed_divide(-4, 2)

(-2, 0)

#### two versions of the same code

In [ ]:
for i in range(num_bits):
    bit       = (numerator & 0x80000000) >> 31 # see above
    numerator = (numerator << 1) & 0xFFFFFFFF  # see above
    remainder = (remainder << 1) | bit         # see above
    t         = remainder - divisor            # 
    q         = t >= 0                         # check if result of subtraction has gone below zero *2
    quotient  = (quotient << 1) | q            # shift up quotient and store negated highest bit of previous division
    if (q): remainder = t                      # if remainder >= divisor: remainder = t

In [ ]:
for i in range(num_bits):
    bit       = (numerator & 0x80000000) >> 31 # see above
    numerator = (numerator << 1) & 0xFFFFFFFF  # see above
    remainder = (remainder << 1) | bit         # see above
    if remainder >= divisor:                   #
        remainder = remainder - divisor        # 
        quotient  = (quotient << 1) | 1        # shift in a 1
    else:                                      #
        quotient  = (quotient << 1)            # shift in a zero

#### Apply this to my problem

The trick we're using here is that in binary long division, producing the result is an iterative process, which takes the bits of the numerator one after the other from most to least significant bit. This bit is then shifted into the least significant bit of the remainder, which is then compared to the divisor, and depending on the result, a one or a zero is shifted into the quotient, while the remainder is reduced back down below the divisor by subtraction.  
We only need a single bit from the numerator for each iteration, and we just so happen to shift the numerator up by one bit each time we want to check the next power of two. 

We continuously divide a power of two $x := 2^{n}$ by a constant multiple of ten $d := 10^{m}$; $x+1 := \frac{x}{d}$  
Each iteration is the same as taking the first number, and dividing it by $d = 10^{mi}$ where $i$ is the $i$th number.  
By saving the intermediate results from the previous iteration, this division can be masively accelerated, since we can continue the division where it left off.  

In [2]:
def ContinueDivision(Quotient, Remainder, Divisor, Bit):
    Remainder = (Remainder << 1) | Bit
    if Remainder >= Divisor:
        Remainder = Remainder - Divisor
        Quotient  = (Quotient << 1) | 1
    else:
        Quotient  = (Quotient << 1)
    return Quotient, Remainder

In [33]:
def ContinueDivision(Quotient, Remainder, Divisor):
    Remainder = (Remainder << 1) # Shift in a zero, since we know that the last bit of 2^x will always be a zero.
    if Remainder >= Divisor:
        Remainder = Remainder - Divisor
        Quotient  = (Quotient << 1) | 1
    else:
        Quotient  = (Quotient << 1)
    return Quotient, Remainder

In [3]:
Number  = 1<<100000; Number, bin(Number)
Number2 = Number << 1

In [4]:
Divisor = 10**399; # Divisor, bin(Divisor)

In [5]:
Quotient  = Number  // Divisor; # Quotient, bin(Quotient)
Quotient2 = Number2 // Divisor

In [6]:
Remainder = Number % Divisor; # Remainder, bin(Remainder)
Remainder2 = Number2 % Divisor

In [35]:
# We can use this to accelerate our algorithm
((Number // Divisor) // Divisor) == Number // (Divisor * Divisor)

True

In [38]:
Quotient2 == ContinueDivision(Quotient, Remainder, Divisor)[0]

True

In [52]:
%%timeit
Q, R = Number // Divisor, Number % Divisor

410 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
Q, R = ContinueDivision(Quotient, Remainder, Divisor)

In [ ]:
def count_sixes_naive(number, divisor):
    six_counter     = 0
    six_counter_max = 0
    while number > 0:
        digit    = number % 10
        number //= divisor
        if digit == 6:
            six_counter    += 1
            six_counter_max = max(six_counter_max, six_counter)
        else:
            six_counter = 0
    return six_counter_max

| $2^{i-1}$ | ${2^i}$ |
|- | - |
| - | $\frac{2^i}{10^{399*1}} $ |
| - | result 2 asdasdsd |
| result 1 | result 2 |
| result 1| - |

In [157]:
profiler = LineProfiler()
@profiler
def count_sixes_new(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        # TODO: Could divide first, which would allow us to remove the (j+1) from the divisor.
        # TODO: use continued division for this as well? Or can't we just use the remainder instead of the number?
        Digit = N % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        try:
            # take last bit from previous result of the current i (N),
            #                 not current result of the previous i (Q),
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
    return SixCounterMax

In [147]:
profiler = LineProfiler()
@profiler
def count_sixes(Number, Divisors, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    N = Number
    while N > 0:
        # TODO: Could divide first, which would allow us to remove the (j+1) from the divisor.
        # TODO: use continued division for this as well?
        Digit = N % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        try:
            # TODO: remove divisors from caching, and instead grab last bit from the numerator
            D    = Divisors[j]
            N, R = Cache[j]
            # N, R = ContinueDivision(Q, R, D) # inline
            R = (R << 1)
            if R >= D:
                R = R - D
                N = (N << 1) | 1
            else:
                N = (N << 1)
        except KeyError:
            D = 10**(399*(j+1))
            N = Number // D
            R = Number  % D
            Divisors[j] = D
        Cache[j] = N, R
        j += 1
    return SixCounterMax

In [ ]:
# OLD

In [219]:
%%time
i        = 400_000
Number   = 1<<(i - 1)
print('Building Caches...')
Divisors = {}
Cache    = {}
count_sixes(Number, Divisors, Cache)
Number = Number << 1
print('Done!')

Building Caches...
Done!
CPU times: total: 21 s
Wall time: 21 s


In [220]:
len(Cache)

302

In [253]:
%%time
count_sixes(Number, Divisors, Cache)
i      += 1
Number  = Number<<1

CPU times: total: 15.6 ms
Wall time: 16 ms


In [254]:
# Verify cache
assert Number == 1<<i
assert len(Divisors) == len(Cache)
assert Divisors[0] == 10**399
assert (Number>>1) // Divisors[0] == Cache[0][0]
assert (Number>>1)  % Divisors[0] == Cache[0][1]
j = len(Cache) - 1
assert (Number>>1) // Divisors[j] == Cache[j][0] == 0
assert (Number>>1)  % Divisors[j] == Cache[j][1]
assert j+1 not in Cache
del j

In [151]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.0144448 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_4028\2519841460.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(Number, Divisors, Cache):
     4         1          6.0      6.0      0.0      SixCounter    = 0
     5         1          3.0      3.0      0.0      SixCounterMax = 0
     6         1          2.0      2.0      0.0      j = 0
     7         1          2.0      2.0      0.0      N = Number
     8       303        857.0      2.8      0.6      while N > 0:
     9                                                   # TODO: Could divide first, which would allow us to remove the (j+1) from the divisor.
    10                                                   # TODO: use continued division for this as well?
    11       302      62965.0    208.5     43.6          

In [164]:
Cache[300][0] == Cache2[300][0]

True

In [ ]:
## NEW

In [153]:
%%time
i        = 400_000
Number   = 1<<(i - 1)
print('Building Cache...')
Divisor = 10**399
Cache2   = {}
count_sixes_new(Number, Divisor, Cache2)
Number = Number << 1
print('Done!')

Building Cache...
Done!
CPU times: total: 297 ms
Wall time: 278 ms


In [189]:
%%time
count_sixes_new(Number, Divisor, Cache2)
i      += 1
Number  = Number<<1

CPU times: total: 15.6 ms
Wall time: 9.97 ms


In [156]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.272942 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_4028\2359129038.py
Function: count_sixes_new at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes_new(N, D, Cache):
     4         1          9.0      9.0      0.0      SixCounter    = 0
     5         1          3.0      3.0      0.0      SixCounterMax = 0
     6         1          3.0      3.0      0.0      j = 0
     7       303       1074.0      3.5      0.0      while N > 0:
     8                                                   # TODO: Could divide first, which would allow us to remove the (j+1) from the divisor.
     9                                                   # TODO: use continued division for this as well? Or can't we just use the remainder instead of the number?
    10       302      70070.0    232.0      2.6          Digit = N %

In [160]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.0092632 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_4028\2359129038.py
Function: count_sixes_new at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes_new(N, D, Cache):
     4         1          8.0      8.0      0.0      SixCounter    = 0
     5         1          3.0      3.0      0.0      SixCounterMax = 0
     6         1          5.0      5.0      0.0      j = 0
     7       303        932.0      3.1      1.0      while N > 0:
     8                                                   # TODO: Could divide first, which would allow us to remove the (j+1) from the divisor.
     9                                                   # TODO: use continued division for this as well? Or can't we just use the remainder instead of the number?
    10       302      63279.0    209.5     68.3          Digit = N 

In [190]:
# Verify cache
assert Number  == 1<<i
assert Divisor == 10**399
assert (Number>>1) // Divisor == Cache2[0][0]
assert (Number>>1)  % Divisor == Cache2[0][1]
j            = len(Cache2) - 1
j0thDivision = (Number>>1) // (10**(399*(j  )))
j1thDivision = (Number>>1) // (10**(399*(j+1)))
assert j0thDivision != 0
assert j1thDivision           == Cache2[j][0] == 0
assert j0thDivision % Divisor == Cache2[j][1]
assert j+1 not in Cache
del j, j0thDivision, j1thDivision

In [ ]:
Cache = {}

In [68]:
Number << 1 == 1<<100_001

True

In [143]:
# %%time
Number = Number << 1
N = Number
i = 0
while N > 0:
    Digit   = N % 10
    print(Digit)
    Divisor = Ten[i]
    if i in Cache:
        Q, R = Cache[i]
        N, R = ContinueDivision(Q, R, Divisor)
    else:
        print('Cache miss!')
        N = Number // Divisor
        R = Number % Divisor
    Cache[i] = (N, R)
    i += 1

2
7
3
0
9
7
1
3
4
4
6
5
2
7
8
9
3
3
3
8
7
5
6
6
0
2
9
7
0
1
7
4
3
5
6
2
1
4
9
8
3
6
7
0
6
4
4
5
9
3
8
2
7
7
6
4
7
0
3
5
5
2
6
2
9
2
3
3
9
5
6
5
3
8
1
5


In [86]:
Number // Ten[1] == Cache[1][0]

True

In [34]:
zr = (zr << 1)
if zr >= y:
    zr = zr - y
    z  = (z << 1) | 1
else:
    z  = (z << 1)

In [ ]:
for i in range(num_bits):
    bit       = (numerator & 0x80000000) >> 31 # see above
    numerator = (numerator << 1) & 0xFFFFFFFF  # see above
    remainder = (remainder << 1) | bit         # see above
    if remainder >= divisor:
        remainder = remainder - divisor            # 
        quotient  = (quotient << 1) | 1            # shift in a 1
    else:
        quotient  = (quotient << 1)                # shift in a zero

In [35]:
z, zr

(507060240091291760598, 6812821504)

In [37]:
(1<<102) // (10**10)

507060240091291760598

In [40]:
bin((1<<100) // (10**10) // (10**10))

'0b1011110011100101000010000110010010'

In [41]:
bin((1<<101) // (10**10) // (10**10))

'0b10111100111001010000100001100100100'